# CAMBIO
Explore creating phenopackts from CAMBIO data (prototype, not for final analysis!)

In [1]:
from cambio import Cambio, Cambio2Phenopacket
from pathlib import Path

In [6]:
input_file = "/Users/hannes/Documents/cambio-analysis/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.csv"
input_file = "/Users/robin/Desktop/HPOstuff/2025_07_24_CAMBio_Tabelle.txt"
input_path = Path(input_file).resolve()
cambio = Cambio(cambio_path=input_path)

/Users/robin/GIT/cambio-analysis/src/cambio/cambio.py:179: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [7]:
cambio2ppkt = Cambio2Phenopacket(df=cambio._df_clean)

In [8]:
ppkt_d = cambio2ppkt.get_ppkt_d()
print(f"Got {len(ppkt_d)} phenopackets")

Got 96 phenopackets


In [20]:
from google.protobuf.json_format import MessageToJson
import phenopackets as PPKt
from pathlib import Path
notebook_dir = Path().resolve()
target_dir = notebook_dir / "phenopackets"
target_dir.mkdir(exist_ok=True)



def output_ppkt(ppkt: PPKt.Phenopacket):
    json_string = MessageToJson(ppkt)
    #print(json_string)
    fname = f"individual_{ppkt.id}.json"
    outpath = target_dir / fname
    fh = open(outpath, "wt")
    fh.write(json_string)
    fh.close()

for k,ppkt in ppkt_d.items():
    output_ppkt(ppkt=ppkt)


In [10]:
cambio2ppkt.add_demographic_data()
cambio2ppkt.add_hpo_columns()

In [11]:
for k, v in ppkt_d.items():
    #print(f"k={k} v={v}")
    output_ppkt(v)

{
  "id": "1",
  "subject": {
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P79Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0000608",
        "label": "Macular degeneration"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0031609",
        "label": "Geographic atrophy"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0004818",
        "label": "Paroxysmal nocturnal hemoglobinuria"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0000822",
        "label": "Hypertension"
      }
    },
    {
      "type": {
        "id": "HP:0001370",
        "label": "Rheumatoid arthritis"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0100665",
        "label": "Angioedema"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0002511",
        "label": "Alzheimer dis

In [12]:
# Phenopackets neu aufbauen
cambio2ppkt = Cambio2Phenopacket(df=cambio._df_clean)
cambio2ppkt.add_demographic_data()
cambio2ppkt.add_hpo_columns()

# Prüfen, ob Hypertension (HP:0000822) enthalten ist
hits = []
for pid, pkt in cambio2ppkt.get_ppkt_d().items():
    for feat in getattr(pkt, "phenotypic_features", []):
        if getattr(feat.type, "id", None) == "HP:0000822":
            hits.append({"PatientID": pid, "Excluded": getattr(feat, "excluded", None)})

print(f"Hypertension gefunden bei {len(hits)} Patienten")
import pandas as pd
if hits:
    display(pd.DataFrame(hits).sort_values("PatientID"))

Hypertension gefunden bei 83 Patienten


,PatientID,Excluded
0,1,False
8,10,False
80,100,False
81,101,True
82,102,False
...,...,...
75,95,False
76,96,False
77,97,True
78,98,False


In [13]:
import json
from google.protobuf.json_format import MessageToDict

pid = next(iter(cambio2ppkt.get_ppkt_d()))
pkt = cambio2ppkt.get_ppkt_d()[pid]
print(json.dumps(MessageToDict(pkt), indent=2))

{
  "id": "1",
  "subject": {
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P79Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0000608",
        "label": "Macular degeneration"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0031609",
        "label": "Geographic atrophy"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0004818",
        "label": "Paroxysmal nocturnal hemoglobinuria"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0000822",
        "label": "Hypertension"
      }
    },
    {
      "type": {
        "id": "HP:0001370",
        "label": "Rheumatoid arthritis"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0100665",
        "label": "Angioedema"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0002511",
        "label": "Alzheimer dis